In [1]:
import folium 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from time import sleep
from folium import plugins
import math
import branca.colormap as cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from itertools import chain 
import geojson

In [2]:
#store the token and url we need for a satellite tile on our map in variables
token = 'pk.eyJ1IjoiYXNobmFrMDMiLCJhIjoiY2s5OTdiYW1wMXBhbDNlcGcwY2plcjhuMyJ9.gMhq8HNZMti-CWHXP3-lTA'
tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

In [3]:
#read datasets and assign to df dataframes
data_file_path = '/Users/ashnakhetan/Desktop/Ashna/ConnectedCars/validation/visualization_1_time.csv'
df = pd.read_csv(data_file_path)

data_file_path_2 = '/Users/ashnakhetan/Desktop/Ashna/ConnectedCars/validation/visualization_2_time.csv'
df2 = pd.read_csv(data_file_path_2)

data_file_path_3 = '/Users/ashnakhetan/Desktop/Ashna/ConnectedCars/validation/visualization_3_time.csv'
df3 = pd.read_csv(data_file_path_3)

data_file_path_4 = '/Users/ashnakhetan/Desktop/Ashna/ConnectedCars/validation/visualization_4_time.csv'
df4 = pd.read_csv(data_file_path_4)

data_file_path_5 = '/Users/ashnakhetan/Desktop/Ashna/ConnectedCars/validation/visualization_5_time.csv'
df5 = pd.read_csv(data_file_path_5)

#create a separate array for aggressiveness values
agS = df['aggresiveness score']
agS2 = df2['aggresiveness score']
agS3 = df3['aggresiveness score']
agS4 = df4['aggresiveness score']
agS5 = df5['aggresiveness score']

aggMin = min(chain(agS, agS2, agS3, agS4, agS5))
aggMax = max(chain(agS, agS2, agS3, agS4, agS5))

#define the aggression range value needed for the color function
aggRange = [aggMin,aggMax]

aggRange

[0.0, 42.45009935436849]

In [4]:
#getColor function is used to decide the color of the lines based on aggression
import matplotlib
# this function takes an agg_range argument related to the range of 
# aggressiveness, then it will calculate its "strength" in a percentage value
# and will go from yellow(RGB)(255,255, 51) to red(255,51,51)
#agg_range=[19.414,60]
def getColor(agg, agg_range):
    yellowGvalue = 1
    redGvalue = .2
    aggPercentage = 1-(agg-agg_range[0])/(agg_range[1]-agg_range[0])
    greenValue = (yellowGvalue-redGvalue)*aggPercentage+redGvalue
    # we define the color based on the greenValue, 
    # red and blue values are 255 and 51, it will scale to yellow as
    # the green value changes
    return matplotlib.colors.to_hex([1,greenValue,.2])

#color_icon
#getColor(20.0,agg_range)
getColor(aggMin,aggRange)
getColor(aggMax,aggRange)

'#ff3333'

In [5]:
#create map with satellite tile
sat = folium.Map(
    location=[42.3053914, -83.6918559], zoom_start = 18 , tiles=tileurl, 
    attr='Mapbox')

In [6]:
#append coordinates, times, colors, and popup content to a consolidated array for each dataset
linesScale=[]
linesScale2=[]
linesScale3=[]
linesScale4=[]
linesScale5=[]

def createLineSet(df, agS, aggRange, lines):
    for i in range(0,len(df)-1):
        
        lines.append(
        {
            'coordinates': [
                [df.longitude[i], df.latitude[i]],
                [df.longitude[i+1], df.latitude[i+1]],
            ],
            'dates': [
                df.time[i], df.time[i+1]],
             'color': getColor(agS[i], aggRange),
             'popup': [df.longitude[i+1], df.latitude[i+1]],
             'aggression': agS[i]

        })

In [7]:
#run the createLineSet function for each dataset
createLineSet(df, agS, aggRange, linesScale)
createLineSet(df2, agS2, aggRange, linesScale2)
createLineSet(df3, agS3, aggRange, linesScale3)
createLineSet(df4, agS4, aggRange, linesScale4)
createLineSet(df5, agS5, aggRange, linesScale5)

In [8]:
#add each LineSet to a list called features; features is passed in the TimestampedGeoJSON function
features = []

def addToFeatures(lines):
    for line in lines:
        features.append(
        {
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates': line['coordinates']
            },
            'properties': {
                #popup shows the point's coordinates and its aggression value
                'popup': str(line['popup']) + '\n \n agg: ' + str(line['aggression']),
                'times': line['dates'],
                'style': {
                    'color': line['color'],
                    'weight': 3 
                }
            }
        }  
    )

In [9]:
#add each LineSet to the array
addToFeatures(linesScale)
addToFeatures(linesScale2)
addToFeatures(linesScale3)
addToFeatures(linesScale4)
addToFeatures(linesScale5)

In [10]:
#finally plot the lines in an animation!
plugins.TimestampedGeoJson({
        'type': 'FeatureCollection',
        'features': features,
    }, period='PT1S', add_last_point=False,time_slider_drag_update=True, min_speed=10, max_speed=15
    ).add_to(sat)

In [11]:
#create a legend to add to map
#https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Colormaps.ipynb

linear = cm.LinearColormap(
    [getColor(aggMin, aggRange), getColor(aggMax, aggRange)],
    vmin=0, vmax=100
).add_to(sat)

In [12]:
#print the map!
sat

In [13]:
#save as html file
sat.save('multi1.html')